In [1]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler

import numpy as np
import matplotlib.pyplot as plt 

import time
import os 
import copy 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:",device)

device: cuda


In [3]:
mean = np.array([0.485,0.456,0.406]) #how to get these? similar to fit in keras?
std = np.array([0.299,0.224,0.225])

In [4]:
data_transforms = {'train':transforms.Compose([transforms.RandomResizedCrop(224),
                  transforms.RandomHorizontalFlip(),
                  transforms.ToTensor(),
                  transforms.Normalize(mean,std)]) , 
                  'val': transforms.Compose([transforms.Resize(256),
                  transforms.CenterCrop(224),
                  transforms.ToTensor(),
                  transforms.Normalize(mean,std)]) }

In [5]:
#Dataset

data_dir = "data/hymenoptera_data/"
splits = ['train','val']

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}

In [6]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 244
     Root location: data/hymenoptera_data/train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485 0.456 0.406], std=[0.299 0.224 0.225])
            ),
 'val': Dataset ImageFolder
     Number of datapoints: 153
     Root location: data/hymenoptera_data/val
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.485 0.456 0.406], std=[0.299 0.224 0.225])
            )}

In [7]:
batch_size = 8
dataset_loaders = {x: DataLoader(dataset=image_datasets[x],batch_size=batch_size,shuffle=True,num_workers=0) for x in ['train','val']}


In [8]:
dataset_loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x1dceb31fd90>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x1dcf3a559a0>}

In [9]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}

In [10]:
dataset_sizes

{'train': 244, 'val': 153}

In [11]:
datasets_classes = image_datasets['train'].classes
num_classes = len(datasets_classes)

print("classes:",datasets_classes)


classes: ['ants', 'bees']


In [12]:
def trainModel(model,loss_func,optimiser,learningrate_scheduler,num_epochs=20):
    start_time = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0 

    steps_per_epoch = {x:len(dataset_loaders['train']) for x in ['train','val']}


    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print("_"*14)

        #Each epoch has training phase and validation phase
        for phase in ['train','val']:

            total_loss = 0
            total_correct_preds = 0


            if phase == 'train':
                model.train() #setting model to train mode

                for i,(inputs,labels) in enumerate(dataset_loaders[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    #forward pass
                    y_preds = model(inputs)
                    _,pred_labels = torch.max(y_preds,1)

                    loss = loss_func(y_preds,labels)

                    correct_preds = torch.eq(pred_labels,labels).sum().item()

                    total_loss += loss
                    total_correct_preds += correct_preds

                    #backward pass
                    optimiser.zero_grad()
                    loss.backward()

                    #weight updation
                    optimiser.step()
                    
                 
                learningrate_scheduler.step() #changing learning rate after every epoch   
                train_epoch_loss = total_loss/len(dataset_loaders[phase])
                train_epoch_acc = 100.0*total_correct_preds/dataset_sizes[phase]
                print(f"Epoch {epoch}, Training Loss {train_epoch_loss},Training Acc{train_epoch_acc}")    

            else :
                model.eval() #setting model to evaluation mode
                
                with torch.no_grad():
                    for i,(inputs,labels) in enumerate(dataset_loaders[phase]):
                        inputs = inputs.to(device)
                        labels = labels.to(device)

                        #forward pass
                        y_preds = model(inputs)
                        _,pred_labels = torch.max(y_preds,1)

                        loss = loss_func(y_preds,labels)

                        correct_preds = torch.eq(pred_labels,labels).sum().item()

                        total_loss += loss
                        total_correct_preds += correct_preds

                    val_epoch_loss = total_loss/len(dataset_loaders[phase])
                    val_epoch_acc = 100.0*total_correct_preds/dataset_sizes[phase]
                    print(f"Epoch {epoch}, Val Loss {val_epoch_loss},Val Acc{val_epoch_acc}")
                    #Save the best model
                    if val_epoch_acc>best_acc :
                        best_acc = val_epoch_acc
                        best_model_wts = copy.deepcopy(model.state_dict())
                        print("Model at epoch {epoch} saved")
                    
            
            
    print("Loading the best model state")
    model.load_state_dict(best_model_wts)
    return model



In [13]:
def trainModel1(model,loss_func,optimiser,scheduler,num_epochs):
    best_acc = 0
    best_model_wts = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}")
        print("_"*20)

        #training phase
        
        model.train()

        total_loss = 0
        total_correct_preds = 0

        for i,(inputs,labels) in enumerate(dataset_loaders['train']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            y_preds = model(inputs)
            # print("y preds",y_preds.shape)
            _,labels_preds = torch.max(y_preds,1)
            # print("labels",labels.shape)
            loss = loss_func(y_preds,labels)

            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

            total_loss += loss.item()
            total_correct_preds += torch.eq(labels_preds,labels).sum().item()

            
        scheduler.step()

        epoch_loss = total_loss/len(dataset_loaders['train'])
        epoch_acc = 100.0*total_correct_preds/dataset_sizes['train']

        print(f"Train Loss: {epoch_loss}, Acc: {epoch_acc}")

        #validation phase
        model.eval()

        total_loss = 0
        total_correct_preds = 0

        for i,(inputs,labels) in enumerate(dataset_loaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                y_preds = model(inputs)
                _,labels_preds = torch.max(y_preds,1)
                loss = loss_func(y_preds,labels)

            

            total_loss += loss.item()
            total_correct_preds += torch.eq(labels_preds,labels).sum().item()


        epoch_loss = total_loss/len(dataset_loaders['val'])
        epoch_acc = 100.0*total_correct_preds/dataset_sizes['val']

        print(f"Val Loss: {epoch_loss}, Acc: {epoch_acc}")


        if epoch_acc>best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
    
    model.load_state_dict(best_model_wts)
    return model


In [14]:
def trainModel2(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataset_loaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [15]:
#freezing convolutional blocks, only train the fully connected layer

model = models.resnet18(pretrained=True) #loading the pretrained model
for param in model.parameters(): #freezing the layers
    param.requires_grad = False
    
#change the last FC layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,num_classes)

model = model.to(device)

In [16]:
loss_func = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(),lr=0.001)
#learning schedulers
lr_scheduler1 = lr_scheduler.StepLR(optimiser,step_size=7,gamma=0.1)
#every 7 epochs, learning rate is multiplied by 0.1

trainedModel = trainModel1(model,loss_func,optimiser,lr_scheduler1,20)

Epoch 0
____________________
Train Loss: 0.6662956937666862, Acc: 59.42622950819672
Val Loss: 0.5916550606489182, Acc: 71.24183006535948
Epoch 1
____________________
Train Loss: 0.5818979557483427, Acc: 70.90163934426229
Val Loss: 0.5042338505387306, Acc: 75.81699346405229
Epoch 2
____________________
Train Loss: 0.538584554387677, Acc: 73.77049180327869
Val Loss: 0.45520116910338404, Acc: 83.66013071895425
Epoch 3
____________________
Train Loss: 0.47540173126805213, Acc: 81.55737704918033
Val Loss: 0.4096618965268135, Acc: 81.69934640522875
Epoch 4
____________________
Train Loss: 0.4798110761950093, Acc: 79.50819672131148
Val Loss: 0.3793720968067646, Acc: 84.9673202614379
Epoch 5
____________________
Train Loss: 0.43151563790536696, Acc: 84.01639344262296
Val Loss: 0.3622742109000683, Acc: 87.58169934640523
Epoch 6
____________________
Train Loss: 0.41332523092146845, Acc: 87.29508196721312
Val Loss: 0.3142951488494873, Acc: 90.84967320261438
Epoch 7
____________________
Train Loss

In [17]:
loss_func = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(),lr=0.001)
#learning schedulers
lr_scheduler1 = lr_scheduler.StepLR(optimiser,step_size=7,gamma=0.1)
#every 7 epochs, learning rate is multiplied by 0.1

trainedModel = trainModel(model,loss_func,optimiser,lr_scheduler1,20)

Epoch 0/19
______________
Epoch 0, Training Loss 0.3824312090873718,Training Acc87.70491803278688
Epoch 0, Val Loss 0.3288092613220215,Val Acc89.54248366013071
Model at epoch {epoch} saved
Epoch 1/19
______________
Epoch 1, Training Loss 0.389901727437973,Training Acc84.42622950819673
Epoch 1, Val Loss 0.2831222116947174,Val Acc91.50326797385621
Model at epoch {epoch} saved
Epoch 2/19
______________
Epoch 2, Training Loss 0.40261343121528625,Training Acc84.8360655737705
Epoch 2, Val Loss 0.2788170278072357,Val Acc92.81045751633987
Model at epoch {epoch} saved
Epoch 3/19
______________
Epoch 3, Training Loss 0.362733393907547,Training Acc87.29508196721312
Epoch 3, Val Loss 0.2596299946308136,Val Acc92.81045751633987
Epoch 4/19
______________
Epoch 4, Training Loss 0.3588903546333313,Training Acc83.60655737704919
Epoch 4, Val Loss 0.2582765817642212,Val Acc92.81045751633987
Epoch 5/19
______________
Epoch 5, Training Loss 0.34403613209724426,Training Acc86.47540983606558
Epoch 5, Val Los

In [18]:
loss_func = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(),lr=0.001)
#learning schedulers
lr_scheduler1 = lr_scheduler.StepLR(optimiser,step_size=7,gamma=0.1)
#every 7 epochs, learning rate is multiplied by 0.1

trainedModel = trainModel2(model,loss_func,optimiser,lr_scheduler1,20)

Epoch 0/19
----------
train Loss: 0.3641 Acc: 0.8361
val Loss: 0.2340 Acc: 0.9216

Epoch 1/19
----------
train Loss: 0.3619 Acc: 0.8279
val Loss: 0.2441 Acc: 0.9346

Epoch 2/19
----------
train Loss: 0.3117 Acc: 0.8811
val Loss: 0.2292 Acc: 0.9216

Epoch 3/19
----------
train Loss: 0.3604 Acc: 0.8443
val Loss: 0.2339 Acc: 0.9412

Epoch 4/19
----------
train Loss: 0.3387 Acc: 0.8443
val Loss: 0.2298 Acc: 0.9477

Epoch 5/19
----------
train Loss: 0.3805 Acc: 0.8361
val Loss: 0.2266 Acc: 0.9412

Epoch 6/19
----------
train Loss: 0.3130 Acc: 0.9016
val Loss: 0.2269 Acc: 0.9346

Epoch 7/19
----------
train Loss: 0.2768 Acc: 0.9180
val Loss: 0.2249 Acc: 0.9412

Epoch 8/19
----------
train Loss: 0.2987 Acc: 0.8852
val Loss: 0.2237 Acc: 0.9477

Epoch 9/19
----------
train Loss: 0.3181 Acc: 0.8730
val Loss: 0.2181 Acc: 0.9346

Epoch 10/19
----------
train Loss: 0.2964 Acc: 0.8811
val Loss: 0.2221 Acc: 0.9412

Epoch 11/19
----------
train Loss: 0.2911 Acc: 0.8893
val Loss: 0.2201 Acc: 0.9150

Ep